In [1]:
import csv
from datetime import datetime
from elasticsearch import Elasticsearch
import json


In [2]:
# settings
es = Elasticsearch([{'host': '127.0.0.1', 'port': 9292}])
indexname = "de70mlcc"
importfile = "C:/Users/hitec_c/Documents/finnews_dep_wft.txt"


In [ ]:
# read docs use counter as additional id to start and stop pushing to elasticsearch
docs = []

# start and end rows are included in the dataset
start = 25000
end = 25000

with open(importfile, newline='', encoding="UTF-8") as f:
    reader = csv.reader(f, delimiter="\t", quoting=csv.QUOTE_NONE)
    counter = 0
    for row in reader:
        if counter >= start:
            ############### parse
            josi = row[0]
            jos = josi.split("<>")
            #print("jos", jos)
            bimsi = row[1]
            bims = bimsi.split("<>")
            #print("bims", bims)
            source = row[2]
            #print("source", source)
            sentence = row[3]
            #print("sentence", sentence)
            time_slice = row[4]
            #print("time_slice", time_slice)
        
            ######### index in es
            doc = {
                'jos': jos,
                'bims': bims,
                'sentence': sentence,
                'source': source,
                'date': time_slice,
                'time_slice': time_slice
            }
            es.index(index=indexname, body=doc)
        
        ##### count and break
        if counter >=end:
            break
        counter += 1

In [11]:
# test query
jo = "Vorstand#NN"
bim = "Berlin#NE#"
res = es.search(index=indexname, 
            body={
                "query": {
                    "bool":{
                        "must":[{
                            "match":{
                                "jos":  
                                {"query": jo,
                                 "operator": "and"
                                }
                            }
                        },{
                            "match": {
                                "bims":
                                {"query": bim,
                                 "operator": "and"
                                }
                            }
                        }]
                    }
                }
            }
        )
#print(res)
ret_set = set()
for hit in res["hits"]["hits"]:
    text = hit["_source"]["date"][:10]+ " [" + str(hit["_source"]["time_slice"])+ "] : " + hit["_source"]["sentence"] + " [" + hit["_source"]["source"] + "] "
    print(text)
    jos = hit["_source"]["jos"]
    print(jos)
    bims = hit["_source"]["bims"]
    print(bims)

1995-1999 [1995-1999] : Die Gesellschaft wird durch ihren alleinigen Vorstand Volkswirt Dr. Hans-Günter Sanftenberg, Berlin, vertreten. [Berliner Zeitung 1995] 
['Gesellschaft#NN', 'Gesellschaft#NN', 'werden#VA', 'werden#VA', 'ihr#PPOSAT', 'alleiniger#ADJA', 'Vorstand#NN', 'Vorstand#NN', 'Vorstand#NN', 'Vorstand#NN', 'Volkswirt#NE', 'Volkswirt#NE', 'Dr.#NN', 'Hans-Günter#NE', 'Sanftenberg#NE', 'Sanftenberg#NE', 'Sanftenberg#NE', 'Sanftenberg#NE', 'Sanftenberg#NE', '--#$,', 'Berlin#NE', 'Berlin#NE', '--#$,', 'vertreten#VV', 'vertreten#VV', 'vertreten#VV', '--#$.']
['Gesellschaft#NN@der#ART#NK', 'Gesellschaft#NN@werden#VA#-SB', 'werden#VA@Gesellschaft#NN#SB', 'werden#VA@vertreten#VV#OC', 'ihr#PPOSAT@Vorstand#NN#-NK', 'alleiniger#ADJA@Vorstand#NN#-NK', 'Vorstand#NN@ihr#PPOSAT#NK', 'Vorstand#NN@vertreten#VV#-MO_durch', 'Vorstand#NN@Volkswirt#NE#NK', 'Vorstand#NN@alleiniger#ADJA#NK', 'Volkswirt#NE@Vorstand#NN#-NK', 'Volkswirt#NE@Sanftenberg#NE#CC', 'Dr.#NN@Sanftenberg#NE#-PNC', 'Hans-Günter

In [12]:
# test query
jo = "Vorstand#NN"
bim = "Berlin#NE-PAR"
jobim = jo + "@" + bim
res = es.search(index=indexname, 
            body={
                "query": {
                  "match":{
                      "bims":
                          {"query": jobim}
                            }
                        }
                }
            )
#print(res)
ret_set = set()
for hit in res["hits"]["hits"]:
    text = hit["_source"]["date"][:10]+ " [" + str(hit["_source"]["time_slice"])+ "] : " + hit["_source"]["sentence"] + " [" + hit["_source"]["source"] + "] "
    print(text)
    jos = hit["_source"]["jos"]
    print(jos)
    bims = hit["_source"]["bims"]
    print(bims)

1995-1999 [1995-1999] : Diese Regelung, erfunden um den CDU-Fraktionsvorsitzenden und Vorstand der Bankgesellschaft Berlin, Klaus Landowsky ("Lex Landowsky"), an die Leine zu legen, ist, mit Verlaub gesagt, großer Unsinn. [Berliner Zeitung 1995] 
['Regelung#NN', 'Regelung#NN', 'Regelung#NN', 'Regelung#NN', '--#$,', 'erfinden#VV', 'erfinden#VV', 'erfinden#VV', 'der#ART', 'Cdu-Fraktionsvorsitzender#NN', 'Cdu-Fraktionsvorsitzender#NN', 'Cdu-Fraktionsvorsitzender#NN', 'Cdu-Fraktionsvorsitzender#NN', 'Cdu-Fraktionsvorsitzender#NN', 'Vorstand#NN', 'Vorstand#NN', 'der#ART', 'Bankgesellschaft#NN', 'Bankgesellschaft#NN', 'Bankgesellschaft#NN', 'Berlin#NE', 'Berlin#NE', 'Berlin#NE', '--#$,', 'Klaus#NE', 'Landowsky#NE', 'Landowsky#NE', 'Landowsky#NE', 'Landowsky#NE', '--#$(', '--#NE', 'Lex#NE', 'Landowsky#NE', '--#NE', '--#NE', '--#NE', '--#NE', '--#NE', '--#$(', '--#$(', '--#$,', 'der#ART', 'Leine#NN', 'Leine#NN', 'zu#PTKZU', 'legen#VV', 'legen#VV', 'legen#VV', 'legen#VV', '--#$,', 'sein#VA', 's

In [ ]:
sa = "[1999, 2000]"
year1 = sa[1:5]
year2 = sa[-5:-1]
ch = year1+"-"+year2
print(ch)
